In [61]:
import sys
sys.path.append('../')
from src import load_dataset

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, strip_accents_unicode
from nltk.stem.snowball import FrenchStemmer
from wordcloud import WordCloud

In [10]:
X, y = load_dataset()

In [133]:
stemmer = FrenchStemmer()
stopwords_list = stopwords.words('french')[:66] #on cut avant les différentes formes du verbe être/avoir qui seront handle par le stemmer

baseVectorizer = CountVectorizer(strip_accents="ascii", stop_words=stopwords_list)
preprocess = baseVectorizer.build_preprocessor()
analyzer = baseVectorizer.build_analyzer()
def mixed_analyzer(doc):
    """
    Stem avec nltk + regarde les stopwords
    """
    preprocess_doc = preprocess(doc)
    tokenized_list = analyzer(preprocess_doc)
    return [stemmer.stem(w) for w in tokenized_list]
    # return tokenized_list

In [134]:
vectorizer = CountVectorizer(
    analyzer=mixed_analyzer,
    )
X_transformed = vectorizer.fit_transform(X)

In [135]:
words = vectorizer.get_feature_names_out()
words[400:500]

array(['absenc', 'absent', 'absente', 'absolu', 'absolus', 'absolut',
       'absorb', 'absorpt', 'absten', 'abstent', 'abstention', 'abstenu',
       'abstract', 'abstrair', 'abstrait', 'absurd', 'absurdit', 'abuj',
       'abus', 'abym', 'acad', 'academ', 'academicien', 'acadien',
       'acapulco', 'accabl', 'accablent', 'accapar', 'acce', 'acced',
       'accedent', 'accederent', 'accel', 'acceler', 'accelere',
       'accelerent', 'accent', 'accentu', 'accentue', 'accentuent',
       'accept', 'accepte', 'acceptent', 'accepton', 'access',
       'accessibilit', 'accessibl', 'accessoir', 'accident', 'accidentel',
       'acclam', 'acclimat', 'accol', 'accommod', 'accommodent',
       'accomod', 'accompagn', 'accompagne', 'accompagnent',
       'accompagnon', 'accompl', 'accord', 'accorde', 'accordent',
       'accordon', 'accost', 'accouch', 'accourt', 'accredit', 'accroch',
       'accrochag', 'accroche', 'accroiss', 'accroissement',
       'accroissent', 'accroit', 'accroitr', 'a

# Prediction

In [139]:
import matplotlib.pyplot as plt
plt.plot(y, kind=pie)

NameError: name 'pie' is not defined

In [136]:
from sklearn.model_selection import train_test_split

# with a seed to enhence reproducibility
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.4, random_state=0) 

In [141]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix


#Naïve Bayes
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)


#Logistic Regression
lr_clf = LogisticRegression(random_state=0, solver='lbfgs',max_iter=100,n_jobs=-1)
lr_clf.fit(X_train, y_train)

#Linear SVM
svm_clf = LinearSVC(random_state=0, tol=1e-5)
svm_clf.fit(X_train, y_train)

pred_nb = nb_clf.predict(X_test)
pred_lr = lr_clf.predict(X_test)
pred_svm = svm_clf.predict(X_test)


print(f"Naïve Bayes accuracy: {confusion_matrix(y_test, pred_nb)}")
print(f"Logistic Regression accuracy: {confusion_matrix(y_test, pred_lr)}")
print(f"SVM accuracy: {confusion_matrix(y_test, pred_svm)}")

/home/charles/M1-S2-DAC/RITAL/TAL/Projet/RITAL-Projet1/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Naïve Bayes accuracy: [[ 1217  1781]
 [  900 19068]]
Logistic Regression accuracy: [[ 1061  1937]
 [  463 19505]]
SVM accuracy: [[ 1223  1775]
 [  887 19081]]


/home/charles/M1-S2-DAC/RITAL/TAL/Projet/RITAL-Projet1/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
